![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/resource-deployment/images/microsoft-small-logo.png)
 
## Create Azure Kubernetes Service cluster and deploy SQL Server 2019 Big Data Cluster
 
This notebook walks through the process of creating a new Azure Kubernetes Service cluster first, and then deploys a <a href="https://docs.microsoft.com/sql/big-data-cluster/big-data-cluster-overview?view=sqlallproducts-allversions">SQL Server 2019 Big Data Cluster</a> on the newly created AKS cluster.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* The **Required information** will check and prompt you for password if it is not set in the environment variable. The password will be used to access the cluster controller, SQL Server, and Knox.

<span style="color:red"><font size="3">Please press the "Run all" button to run the notebook</font></span>

### **Prerequisites**
Ensure the following tools are installed and added to PATH before proceeding.

|Tools|Description|Installation|
|---|---|---|
|Azure CLI |Command-line tool for managing Azure services. Used to create AKS cluster | [Installation](https://docs.microsoft.com/cli/azure/install-azure-cli?view=azure-cli-latest) |
|kubectl | Command-line tool for monitoring the underlying Kubernetes cluster | [Installation](https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management) |
|azdata | Command-line tool for installing and managing a Big Data Cluster |[Installation](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-install-azdata?view=sqlallproducts-allversions) |

### **Setup**

In [2]:
import pandas,sys,os,json,html,getpass,time
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')
def run_command(command):
    print("Executing: " + command)
    !{command}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')

### **Set variables**
Generated by Azure Data Studio using the values collected in the Deploy Big Data Cluster wizard

In [3]:
azure_subscription_id = 'b7e65cb3-9829-44ea-9a90-6250cc442b3b'
azure_region = 'southafricanorth'
azure_resource_group = 'rg-sqlbdc'
azure_vm_size = 'Standard_B8ms'
azure_vm_count = '3'
aks_cluster_name = 'kubesqlbdc-cluster'
mssql_cluster_name = 'sqlbdc-cluster'
mssql_username = 'admin'
mssql_auth_mode = 'basic'
bdc_json = '{"apiVersion":"v1","metadata":{"kind":"BigDataCluster","name":"sqlbdc-cluster"},"spec":{"resources":{"nmnode-0":{"spec":{"replicas":1}},"sparkhead":{"spec":{"replicas":1}},"zookeeper":{"spec":{"replicas":0}},"gateway":{"spec":{"replicas":1,"endpoints":[{"name":"Knox","serviceType":"LoadBalancer","port":30443}]}},"appproxy":{"spec":{"replicas":1,"endpoints":[{"name":"AppServiceProxy","serviceType":"LoadBalancer","port":30778}]}},"master":{"metadata":{"kind":"Pool","name":"default"},"spec":{"type":"Master","replicas":1,"endpoints":[{"name":"Master","serviceType":"LoadBalancer","port":31433},{"name":"MasterSecondary","serviceType":"NodePort","port":31436}],"settings":{"sql":{"hadr.enabled":"false"}}},"storage":{"data":{"size":"15Gi","className":"default","accessMode":"ReadWriteOnce"},"logs":{"size":"10Gi","className":"default","accessMode":"ReadWriteOnce"}}},"compute-0":{"metadata":{"kind":"Pool","name":"default"},"spec":{"type":"Compute","replicas":1}},"data-0":{"metadata":{"kind":"Pool","name":"default"},"spec":{"type":"Data","replicas":2},"storage":{"data":{"size":"15Gi","className":"default","accessMode":"ReadWriteOnce"},"logs":{"size":"10Gi","className":"default","accessMode":"ReadWriteOnce"}}},"storage-0":{"metadata":{"kind":"Pool","name":"default"},"spec":{"type":"Storage","replicas":2,"settings":{"spark":{"includeSpark":true}}},"storage":{"data":{"size":"15Gi","className":"default","accessMode":"ReadWriteOnce"},"logs":{"size":"10Gi","className":"default","accessMode":"ReadWriteOnce"}}}},"services":{"sql":{"resources":["master","compute-0","data-0","storage-0"]},"hdfs":{"resources":["nmnode-0","zookeeper","storage-0","sparkhead"],"settings":{}},"spark":{"resources":["sparkhead","storage-0"],"settings":{"spark-defaults-conf.spark.driver.cores":"1","spark-defaults-conf.spark.driver.memory":"1664m","spark-defaults-conf.spark.driver.memoryOverhead":"384","spark-defaults-conf.spark.executor.instances":"1","spark-defaults-conf.spark.executor.cores":"2","spark-defaults-conf.spark.executor.memory":"3712m","spark-defaults-conf.spark.executor.memoryOverhead":"384","yarn-site.yarn.nodemanager.resource.memory-mb":"12288","yarn-site.yarn.nodemanager.resource.cpu-vcores":"6","yarn-site.yarn.scheduler.maximum-allocation-mb":"12288","yarn-site.yarn.scheduler.maximum-allocation-vcores":"6","yarn-site.yarn.scheduler.capacity.maximum-am-resource-percent":"0.34"}}}}}'
control_json = '{"apiVersion":"v1","metadata":{"kind":"Cluster","name":"mssql-cluster"},"spec":{"docker":{"registry":"mcr.microsoft.com","repository":"mssql/bdc","imageTag":"2019-CU6-ubuntu-16.04","imagePullPolicy":"Always"},"storage":{"data":{"className":"default","accessMode":"ReadWriteOnce","size":"15Gi"},"logs":{"className":"default","accessMode":"ReadWriteOnce","size":"10Gi"}},"endpoints":[{"name":"Controller","serviceType":"LoadBalancer","port":30080},{"name":"ServiceProxy","serviceType":"LoadBalancer","port":30777}],"settings":{"controller":{"logs.rotation.size":"5000","logs.rotation.days":"7"}}}}'
os.environ["AZDATA_NB_VAR_KUBECTL"] = "C:\\ProgramData\\chocolatey\\bin\\kubectl.exe"
os.environ["PATH"] = os.environ["PATH"] + ";" + "C:\\ProgramData\\chocolatey\\bin;C:\\Program Files (x86)\\Microsoft SDKs\\Azure\\CLI2\\wbin;C:\\Python38\\Scripts"
print('Variables have been set successfully.')


Variables have been set successfully.


### **Check dependencies**

In [4]:
run_command('kubectl version --client=true')
run_command('azdata --version')
run_command('az --version')

Executing: kubectl version --client=true


Client Version: version.Info{Major:"1", Minor:"18", GitVersion:"v1.18.0", GitCommit:"9e991415386e4cf155a24b1da15becaa390438d8", GitTreeState:"clean", BuildDate:"2020-03-25T14:58:59Z", GoVersion:"go1.13.8", Compiler:"gc", Platform:"windows/amd64"}
Successfully executed: kubectl version --client=true
Executing: azdata --version


20.0.1

SQL Server 2019 (15.0.4053)

Python (Windows) 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)]

Python location 'c:\python38\python.exe'

Successfully executed: azdata --version
Executing: az --version


azure-cli                          2.2.0 *

command-modules-nspkg              2.0.3
core                               2.2.0 *
nspkg                              3.0.4
telemetry                          1.0.4

Python location 'C:\Program Files (x86)\Microsoft SDKs\Azure\CLI2\python.exe'
Extensions directory 'C:\Users\niels\.azure\cliextensions'

Python (Windows) 3.6.6 (v3.6.6:4cf1f54eb7, Jun 27 2018, 02:47:15) [MSC v.1900 32 bit (Intel)]

Legal docs and information: aka.ms/AzureCliLegal




Please let us know how we are doing: https://aka.ms/clihats
Successfully executed: az --version


### **Required information**

In [5]:
invoked_by_wizard = "AZDATA_NB_VAR_BDC_ADMIN_PASSWORD" in os.environ
if invoked_by_wizard:
    mssql_password = os.environ["AZDATA_NB_VAR_BDC_ADMIN_PASSWORD"]
else:
    mssql_password = getpass.getpass(prompt = 'SQL Server 2019 Big Data Cluster controller password')
    if mssql_password == "":
        sys.exit(f'Password is required.')
    confirm_password = getpass.getpass(prompt = 'Confirm password')
    if mssql_password != confirm_password:
        sys.exit(f'Passwords do not match.')
print('You can also use the controller password to access Knox and SQL Server.')

You can also use the controller password to access Knox and SQL Server.


### **Login to Azure**

This will open a web browser window to enable credentials to be entered. If this cells is hanging forever, it might be because your Web browser windows is waiting for you to enter your Azure credentials!


In [6]:
run_command(f'az login')

Executing: az login


[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "6778d36c-50b7-4320-8d62-6a5091d6e2e7",
    "id": "a16fbc6b-dc1f-43f7-a30d-3ad9c3760594",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Visual Studio Premium with MSDN",
    "state": "Enabled",
    "tenantId": "6778d36c-50b7-4320-8d62-6a5091d6e2e7",
    "user": {
      "name": "niels.it.berglund@gmail.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "6778d36c-50b7-4320-8d62-6a5091d6e2e7",
    "id": "b7e65cb3-9829-44ea-9a90-6250cc442b3b",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
      }
    ],
    "name": "Microsoft Azure Sponsorship",
    "state": "Enabled",
    "tenantId": "6778d36c-50b7-4320-8d62-6a5091d6e2e7",
    "user": {
      "name": "niels.it.berglund@gmail.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "8e37b28c-0fd9-4dd7-ad44-3ca037f2d9fc"


### **Set active Azure subscription**

In [7]:
if azure_subscription_id != "":
    run_command(f'az account set --subscription {azure_subscription_id}')
else:
    print('Using the default Azure subscription', {azure_subscription_id})
run_command(f'az account show')

Executing: az account set --subscription b7e65cb3-9829-44ea-9a90-6250cc442b3b


Successfully executed: az account set --subscription b7e65cb3-9829-44ea-9a90-6250cc442b3b
Executing: az account show


{
  "environmentName": "AzureCloud",
  "homeTenantId": "6778d36c-50b7-4320-8d62-6a5091d6e2e7",
  "id": "b7e65cb3-9829-44ea-9a90-6250cc442b3b",
  "isDefault": true,
  "managedByTenants": [
    {
      "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
    }
  ],
  "name": "Microsoft Azure Sponsorship",
  "state": "Enabled",
  "tenantId": "6778d36c-50b7-4320-8d62-6a5091d6e2e7",
  "user": {
    "name": "niels.it.berglund@gmail.com",
    "type": "user"
  }
}
Successfully executed: az account show


### **Create Azure resource group**

In [8]:
run_command(f'az group create --name {azure_resource_group} --location {azure_region}')

Executing: az group create --name rg-sqlbdc --location southafricanorth


{
  "id": "/subscriptions/b7e65cb3-9829-44ea-9a90-6250cc442b3b/resourceGroups/rg-sqlbdc",
  "location": "southafricanorth",
  "managedBy": null,
  "name": "rg-sqlbdc",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
Successfully executed: az group create --name rg-sqlbdc --location southafricanorth


### **Create AKS cluster**

In [9]:
run_command(f'az aks create --name {aks_cluster_name} --resource-group {azure_resource_group} --generate-ssh-keys --node-vm-size {azure_vm_size} --node-count {azure_vm_count}')

Executing: az aks create --name kubesqlbdc-cluster --resource-group rg-sqlbdc --generate-ssh-keys --node-vm-size Standard_B8ms --node-count 3


{
  "aadProfile": null,
  "addonProfiles": {
    "KubeDashboard": {
      "config": null,
      "enabled": true,
      "identity": null
    }
  },
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 3,
      "enableAutoScaling": null,
      "enableNodePublicIp": null,
      "maxCount": null,
      "maxPods": 110,
      "minCount": null,
      "name": "nodepool1",
      "nodeLabels": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.16.13",
      "osDiskSizeGb": 128,
      "osType": "Linux",
      "provisioningState": "Succeeded",
      "scaleSetEvictionPolicy": null,
      "scaleSetPriority": null,
      "tags": null,
      "type": "VirtualMachineScaleSets",
      "vmSize": "Standard_B8ms",
      "vnetSubnetId": null
    }
  ],
  "apiServerAccessProfile": null,
  "dnsPrefix": "kubesqlbdc-rg-sqlbdc-b7e65c",
  "enablePodSecurityPolicy": null,
  "enableRbac": true,
  "fqdn": "kubesqlbdc-rg-sqlbdc-b7e65c-874a23c8.hcp.southafricanorth.azmk8s.io",
  "


Finished service principal creation[##################################]  100.0000%


### **Set the new AKS cluster as current context**

In [10]:
run_command(f'az aks get-credentials --resource-group {azure_resource_group} --name {aks_cluster_name} --admin --overwrite-existing')

Executing: az aks get-credentials --resource-group rg-sqlbdc --name kubesqlbdc-cluster --admin --overwrite-existing


Merged "kubesqlbdc-cluster-admin" as current context in C:\Users\niels\.kube\config
Successfully executed: az aks get-credentials --resource-group rg-sqlbdc --name kubesqlbdc-cluster --admin --overwrite-existing


### **Create deployment configuration files**

In [11]:
mssql_target_profile = 'ads-bdc-custom-profile'
if not os.path.exists(mssql_target_profile):
    os.mkdir(mssql_target_profile)
bdcJsonObj = json.loads(bdc_json)
controlJsonObj = json.loads(control_json)
bdcJsonFile = open(f'{mssql_target_profile}/bdc.json', 'w')
bdcJsonFile.write(json.dumps(bdcJsonObj, indent = 4))
bdcJsonFile.close()
controlJsonFile = open(f'{mssql_target_profile}/control.json', 'w')
controlJsonFile.write(json.dumps(controlJsonObj, indent = 4))
controlJsonFile.close()
print(f'Created deployment configuration folder: {mssql_target_profile}')

Created deployment configuration folder: ads-bdc-custom-profile


### **Create SQL Server 2019 Big Data Cluster**

In [12]:
print (f'Creating SQL Server 2019 Big Data Cluster: {mssql_cluster_name} using configuration {mssql_target_profile}')
os.environ["ACCEPT_EULA"] = 'yes'
os.environ["AZDATA_USERNAME"] = mssql_username
os.environ["AZDATA_PASSWORD"] = mssql_password
if os.name == 'nt':
    print(f'If you don\'t see output produced by azdata, you can run the following command in a terminal window to check the deployment status:\n\t{os.environ["AZDATA_NB_VAR_KUBECTL"]} get pods -n {mssql_cluster_name} ')
run_command(f'azdata bdc create -c {mssql_target_profile}')

Creating SQL Server 2019 Big Data Cluster: sqlbdc-cluster using configuration ads-bdc-custom-profile
If you don't see output produced by azdata, you can run the following command in a terminal window to check the deployment status:
	C:\ProgramData\chocolatey\bin\kubectl.exe get pods -n sqlbdc-cluster 
Executing: azdata bdc create -c ads-bdc-custom-profile


The privacy statement can be viewed at:
https://go.microsoft.com/fwlink/?LinkId=853010

The license terms for SQL Server Big Data Cluster can be viewed at:
Enterprise: https://go.microsoft.com/fwlink/?linkid=2104292
Standard: https://go.microsoft.com/fwlink/?linkid=2104294
Developer: https://go.microsoft.com/fwlink/?linkid=2104079


Cluster deployment documentation can be viewed at:
https://aka.ms/bdc-deploy

NOTE: Cluster creation can take a significant amount of time depending on
configuration, network speed, and the number of nodes in the cluster.

Starting cluster deployment.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Waiting for cluster controller to start.
Cluster controller endpoint is available at 

### **Login to SQL Server 2019 Big Data Cluster**

In [13]:
run_command(f'azdata login -n {mssql_cluster_name}')

Executing: azdata login -n sqlbdc-cluster


Logged in successfully to `https://102.37.18.57:30080` in namespace `sqlbdc-cluster`. Setting active context to `sqlbdc-cluster`.
Successfully executed: azdata login -n sqlbdc-cluster


### **Show SQL Server 2019 Big Data Cluster endpoints**

In [14]:
from IPython.display import *
pandas.set_option('display.max_colwidth', -1)
cmd = f'azdata bdc endpoint list'
cmdOutput = !{cmd}
endpoints = json.loads(''.join(cmdOutput))
endpointsDataFrame = pandas.DataFrame(endpoints)
endpointsDataFrame.columns = [' '.join(word[0].upper() + word[1:] for word in columnName.split()) for columnName in endpoints[0].keys()]
display(HTML(endpointsDataFrame.to_html(index=False, render_links=True)))

Description,Endpoint,Name,Protocol
"Gateway to access HDFS files, Spark",https://40.127.13.234:30443,gateway,https
Spark Jobs Management and Monitoring Dashboard,https://40.127.13.234:30443/gateway/default/sparkhistory,spark-history,https
Spark Diagnostics and Monitoring Dashboard,https://40.127.13.234:30443/gateway/default/yarn,yarn-ui,https
Application Proxy,https://102.37.18.92:30778,app-proxy,https
Management Proxy,https://102.37.17.75:30777,mgmtproxy,https
Log Search Dashboard,https://102.37.17.75:30777/kibana,logsui,https
Metrics Dashboard,https://102.37.17.75:30777/grafana,metricsui,https
Cluster Management Service,https://102.37.18.57:30080,controller,https
SQL Server Master Instance Front-End,"102.37.18.98,31433",sql-server-master,tds
HDFS File System Proxy,https://40.127.13.234:30443/gateway/default/webhdfs/v1,webhdfs,https


### **Connect to SQL Server Master instance in Azure Data Studio**
Click the link below to connect to the SQL Server Master instance of the SQL Server 2019 Big Data Cluster.

In [15]:
sqlEndpoints = [x for x in endpoints if x['name'] == 'sql-server-master']
if sqlEndpoints and len(sqlEndpoints) == 1:
    connectionParameter = '{"serverName":"' + sqlEndpoints[0]['endpoint'] + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":' + json.dumps(mssql_username) + ',"password":' + json.dumps(mssql_password) + '}'
    display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to SQL Server Master instance</font></a><br/>'))
    display(HTML('<br/><span style="color:red"><font size="2">NOTE: The SQL Server password is included in this link, you may want to clear the results of this code cell before saving the notebook.</font></span>'))
else:
    sys.exit('Could not find the SQL Server Master instance endpoint.')